# OK Injection wells

There are two types of files: a single file for 2006-2010 that gives annual volumes (API must be constructed from the county codes, etc.)  and a set of yearly files with monthly data in a very different format. The source of the data is: http://occeweb.com/og/ogdatafiles2.htm

Because they are so different, I will handle them as two separate jobs and merge them at the end.  The meta data will also be pulled from these files, the most recent entries taken and then merged with the data.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

metafn = './sources/AllUICWells.xlsx' # This file not used.  Didn't have many early injection wells.

inj2011_datadir = './sources/2011-2018/'
inj2006_datadir = './sources/2006-2010/'
fn2006 = inj2006_datadir+'all_2006-2010uic_1012a.xls'
outdir = './out/'

# Years 2006-2010

In [2]:
def fetch_raw_OK_injection1(fn = fn2006):
    print(f'Processing: {fn}')
    d = pd.read_excel(fn)
    d.API_COUNTY = d.API_COUNTY.astype('str')
    d.API_COUNTY = d.API_COUNTY.str.zfill(3)
    d.API_NUMBER = d.API_NUMBER.astype('str')
    d.API_NUMBER = d.API_NUMBER.str.zfill(5)
    d['API'] = '35'+d.API_COUNTY.str[:] + d.API_NUMBER.str[:] 
    print(f'    length of data from this file: {len(d)}') 
    return d


In [3]:
d06 = fetch_raw_OK_injection1()
d06.columns = ['1','2','WellName','WellNumber','Latitude','Longitude','Year','5','6','Volume','API']
meta06 = d06.copy()  # put aside a copy for preparing metadata to merge back in later

Processing: ./sources/2006-2010/all_2006-2010uic_1012a.xls
    length of data from this file: 53409


In [4]:
d06 = d06.filter(['API','Year','Volume'], axis=1) # keep just these columns
d06['YearMonth'] = 'Y'+d06.Year.astype('str') # Create a variable that will eventually be a column label
d06.head()

,API,Year,Volume,YearMonth
0,3500300026,2007,9540,Y2007
1,3500300026,2008,12236,Y2008
2,3500300026,2009,11825,Y2009
3,3500300026,2010,11863,Y2010
4,3500300163,2006,34783,Y2006


In [5]:
# test = d06.groupby(['API','Yr'],as_index=False).count()
test = d06.sort_values(by=['API','YearMonth'])
test[test.duplicated(subset=['API','YearMonth'])==True].head()

,API,Year,Volume,YearMonth
247,3500720654,2007,144140,Y2007
249,3500720654,2008,219084,Y2008
252,3500720654,2010,256194,Y2010
254,3500720687,2006,217799,Y2006
256,3500720687,2007,91858,Y2007


**There are multiple rows for some wells for a given year.**  In some cases, it is clear they are duplicates, but in others, the volumes are different.  As with the 2011-17 data, we choose the max cell of the repeats to represent the well for that year.  (What these repeats mean, I don't know...)

**There are 32 records in which the year is out of range (between 2011 to 4371).** We remove those records.

In [6]:
len(d06[d06.Year>=2011])

32

In [7]:
d06 = d06[d06.Year<=2010]

In [8]:
d06 = d06.groupby(['API','YearMonth'],as_index=False)['Volume'].max()
pivot06 = d06.pivot(index='API',columns='YearMonth')
pivot06.columns = pivot06.columns.get_level_values(1)
pivot06.head()

YearMonth,Y2006,Y2007,Y2008,Y2009,Y2010
API,,,,,
3500300026,NaN,9540.0,12236.0,11825.0,11863.0
3500300163,34783.0,35890.0,38250.0,42953.0,20120.0
3500320145,412402.0,952745.0,479685.0,149277.0,130934.0
3500320184,0.0,NaN,NaN,NaN,NaN
3500320206,0.0,NaN,NaN,NaN,NaN


# Years 2011 on...

In [9]:
def fetch_raw_OK_injection2(indir=inj2011_datadir):
    # it is assumed that all files in the directory are
    # to be processed.
    flist = os.listdir(indir)
    if len(flist)<1 :
        raise Exception('No files to process!')
    for f in flist:
        if f[-5:] != '.xlsx':
            raise Exception('Some non-excel files present in input directory')
    dlst = []
    for fn in flist:
        print(f'Processing: {fn}')
        d = pd.read_excel(indir+fn)
        d = d.filter(['API','ReportYear',
                      'Jan Vol','Feb Vol','Mar Vol','Apr Vol','May Vol','Jun Vol',
                      'Jul Vol','Aug Vol','Sep Vol','Oct Vol','Nov Vol','Dec Vol']) 
        d = pd.melt(d,id_vars=['API','ReportYear']) # make it a TALL frame
        dlst.append(d)
        print(f'    length of data from this file: {len(d)}') 
    df_inj = pd.concat(dlst,sort=True)
    print(f'Total length of dataframe: {len(df_inj)}')
    return df_inj


In [10]:
d = fetch_raw_OK_injection2()
d.API = d.API.astype('str').str[:10]
d.describe(include='all')

Processing: 2011 1012A UIC volumes.xlsx
    length of data from this file: 118560
Processing: 2012 1012A UIC volumes.xlsx
    length of data from this file: 126000
Processing: 2013 1012A UIC volumes.xlsx
    length of data from this file: 129516
Processing: 2014 1012A UIC volumes.xlsx
    length of data from this file: 137292
Processing: 2015 1012A UIC volumes.xlsx
    length of data from this file: 139092
Processing: 2016 1012A UIC volumes.xlsx
    length of data from this file: 143220
Processing: UIC injection volumes 2017.xlsx
    length of data from this file: 149616
Processing: UIC injection volumes 2018.xlsx
    length of data from this file: 145992
Total length of dataframe: 1089288


,API,ReportYear,value,variable
count,1089288,1.089288e+06,1.089288e+06,1089288
unique,11650,NaN,NaN,12
top,3509324452,NaN,NaN,May Vol
freq,720,NaN,NaN,90774
mean,NaN,2.014662e+03,2.030148e+04,NaN
std,NaN,2.270915e+00,6.249125e+04,NaN
min,NaN,2.011000e+03,-5.000000e+02,NaN
25%,NaN,2.013000e+03,2.000000e+01,NaN
50%,NaN,2.015000e+03,4.000000e+03,NaN
75%,NaN,2.017000e+03,1.802700e+04,NaN


In [11]:
mon = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
       'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
def make_YearMonth(row):
    yr = str(row.ReportYear)
    mo = mon[str(row.variable)[:3]]
    return 'Y'+yr+'M'+mo

d['YearMonth'] = d.apply(lambda x: make_YearMonth(x),axis=1)

In [12]:
d.head()

,API,ReportYear,value,variable,YearMonth
0,3500300026,2011,971.0,Jan Vol,Y2011M01
1,3500300163,2011,1132.0,Jan Vol,Y2011M01
2,3500320145,2011,660.0,Jan Vol,Y2011M01
3,3500320145,2011,660.0,Jan Vol,Y2011M01
4,3500320786,2011,0.0,Jan Vol,Y2011M01


In [13]:
# drop unused columns
trimmed = d.drop(['ReportYear','variable'],axis=1).copy()

# check if there are multiple records for year/month instances
test = trimmed.groupby(['YearMonth','API'],as_index=False).count()
# test.value.unique()
test[test.value>2].head(20)

,YearMonth,API,value
20,Y2011M01,3500321840,3
130,Y2011M01,3500725389,4
188,Y2011M01,3500921057,4
196,Y2011M01,3500930000,3
203,Y2011M01,3501120334,3
210,Y2011M01,3501120940,3
213,Y2011M01,3501120986,3
221,Y2011M01,3501122466,3
226,Y2011M01,3501123263,3
227,Y2011M01,3501123283,3


**Yes, lots of multiple entries for an API/YearMonth combination**  These seem to be due mostly to changes in metadata (in particular, formation name) and the quantity does not change across the repeats.  However, there are some repeats with both METERED and CALCULATED quantities that differ.  Ted talked to the folks at OCC and have concluded that we take the largest of the values across the repeats.


In [14]:
test = trimmed.groupby(['YearMonth','API'],as_index=False).max()
test.head()

,YearMonth,API,value
0,Y2011M01,3500300026,971.0
1,Y2011M01,3500300163,1132.0
2,Y2011M01,3500320145,660.0
3,Y2011M01,3500320786,0.0
4,Y2011M01,3500320929,866.0


Just checking: How many API are not active?


In [15]:
t = trimmed.groupby(['API'],as_index=False)['value'].sum()
print(f'Total number of wells: {len(t)}')
print(f'Number unused wells:   {len(t[t.value==0])}')

Total number of wells: 11650
Number unused wells:   1263


From here, we just pivot to get the data in the orientation that FracTracker needs to post to maps.

In [16]:
pivot2011 = test.pivot(index='API',columns='YearMonth')
pivot2011.columns = pivot2011.columns.get_level_values(1)
pivot2011.head()

YearMonth,Y2011M01,Y2011M02,Y2011M03,Y2011M04,Y2011M05,Y2011M06,Y2011M07,Y2011M08,Y2011M09,Y2011M10,...,Y2018M03,Y2018M04,Y2018M05,Y2018M06,Y2018M07,Y2018M08,Y2018M09,Y2018M10,Y2018M11,Y2018M12
API,,,,,,,,,,,,,,,,,,,,,
3500300026,971.0,986.0,994.0,990.0,963.0,997.0,986.0,0.0,0.0,0.0,...,0.0,0.0,253.0,508.0,354.0,274.0,425.0,0.0,0.0,765.0
3500300163,1132.0,992.0,1296.0,1191.0,1132.0,1191.0,1132.0,1132.0,1132.0,1296.0,...,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0
3500320145,660.0,0.0,3074.0,20266.0,5926.0,0.0,10963.0,12929.0,0.0,20192.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3500320786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3500320929,866.0,313.0,648.0,251.0,237.0,341.0,372.0,372.0,264.0,184.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge the two sets

In [17]:
mg = pd.merge(pivot06,pivot2011,on='API',how='outer',validate='1:1')

In [18]:
mg.tail()

YearMonth,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011M01,Y2011M02,Y2011M03,Y2011M04,Y2011M05,...,Y2018M03,Y2018M04,Y2018M05,Y2018M06,Y2018M07,Y2018M08,Y2018M09,Y2018M10,Y2018M11,Y2018M12
API,,,,,,,,,,,,,,,,,,,,,
3515124522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10350.0,12150.0,11300.0,10908.0,38646.0,54174.0,66700.0,42758.0,55629.0,18780.0
3515322534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,309.0,140.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3515323579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59536.0,135567.0,106658.0,108570.0,170007.0,150257.0,213454.0,194364.0,147499.0,212815.0
3515323593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3515323594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
len(mg)

13864

# Get the metadata

In [20]:
meta06 = meta06.groupby('API',as_index=False)['WellName','WellNumber','Latitude','Longitude'].last()

In [21]:
meta06.tail()

,API,WellName,WellNumber,Latitude,Longitude
11567,3515323353,MCGOU 'A',1-27,36.262634,-99.54862
11568,3515323377,WOODWARD IODINE,45,36.521669,-99.33446
11569,3515323424,CEI PETROLEUM INC.,24-1,36.276494,-99.29852
11570,3515323507,JIMMIE SWD,1-28,36.435910,-99.13650
11571,3515335187,CANFIELD,1,36.412420,-99.38155


In [22]:
def fetch_raw_OK_for_meta(indir=inj2011_datadir):
    # it is assumed that all files in the directory are
    # to be processed.
    flist = os.listdir(indir)
    if len(flist)<1 :
        raise Exception('No files to process!')
    for f in flist:
        if f[-5:] != '.xlsx':
            raise Exception('Some non-excel files present in input directory')
    dlst = []
    for fn in flist:
        print(f'Processing: {fn}')
        d = pd.read_excel(indir+fn)
        d = d.filter(['API','OperatorName','WellName','WellNumber','Lat_Y','Long_X'],axis=1) 
        d.columns = ['API','OperatorName','WellName','WellNumber','Latitude','Longitude']
        dlst.append(d)
        print(f'    length of data from this file: {len(d)}') 
    df_inj = pd.concat(dlst,sort=True)
    print(f'Total length of dataframe: {len(df_inj)}')
    return df_inj

In [23]:
meta11 = fetch_raw_OK_for_meta()
meta11.API = meta11.API.astype('str').str[:10]
meta11 = meta11.groupby('API',as_index=False)['OperatorName','WellName',
                                              'WellNumber','Latitude','Longitude'].last()

Processing: 2011 1012A UIC volumes.xlsx
    length of data from this file: 9880
Processing: 2012 1012A UIC volumes.xlsx
    length of data from this file: 10500
Processing: 2013 1012A UIC volumes.xlsx
    length of data from this file: 10793
Processing: 2014 1012A UIC volumes.xlsx
    length of data from this file: 11441
Processing: 2015 1012A UIC volumes.xlsx
    length of data from this file: 11591
Processing: 2016 1012A UIC volumes.xlsx
    length of data from this file: 11935
Processing: UIC injection volumes 2017.xlsx
    length of data from this file: 12468
Processing: UIC injection volumes 2018.xlsx
    length of data from this file: 12166
Total length of dataframe: 90774


In [24]:

meta11.head()

,API,OperatorName,WellName,WellNumber,Latitude,Longitude
0,3500300026,PHOENIX PETROCORP INC,SE EUREKA UNIT-TUCKER #1,21,36.900324,-98.218260
1,3500300163,SUPERIOR PRODUCTION LLC,CHRISTENSEN,1,36.896636,-98.177720
2,3500320145,CONTINENTAL RESOURCES INC,SINGREE,1,36.504849,-98.433240
3,3500320786,LINN OPERATING LLC,NE CHEROKEE UNIT,85,36.806113,-98.325840
4,3500320929,CHAPARRAL ENERGY LLC,R & H,1,36.962410,-98.519482


Put the metas together

In [25]:
mts = pd.concat([meta06,meta11],sort=True)
mts = mts.groupby('API').last()
mts.head()
print(f'Length of meta {len(mts)}')

Length of meta 13865


In [27]:
final = pd.merge(mts,mg,on='API',how='right',validate='1:1')
final.head()
final.to_excel(outdir+'OK_injection2006-18.xlsx')